In [6]:
import os
import sys
import numpy as np

sys.path.append("/mnt/lareaulab/reliscu/code")

from parse_gtf import *

In [ ]:
psi_data = "SyntheticDataset1_20pcntCells_35SD_200samples_SJ_pseudobulk_min_observed0.25_minPsi0.1_PSI"

In [ ]:
psi = pd.read_csv(f"data/tasic_2018_ALM_STAR_{psi_data}.csv", index_col=0)
top_qval_mods_df = pd.read_csv("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_20pcntCells_35SD_200samples_log2_pseudobulk_PosBC_top_Qval_modules.csv")

### Add gene names to PSI data

In [ ]:
# Parse GTF attribute column
gtf_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/gencode.vM35.annotation.gtf"
gtf = gtf_parse(gtf_file)
gtf_subset = gtf.loc[gtf['feature'].isin(["gene"])]
attrs = gtf_subset["attribute"].apply(extract_attributes)
attrs_df = attrs.apply(pd.Series)
gtf_parsed = pd.concat([gtf_subset.drop(columns=["attribute"]), attrs_df], axis=1)

In [4]:
# Get PSI and GTF data ready to merge on gene IDs
gtf_parsed['gene_id'] = gtf_parsed['gene_id'].str.split(".").str[0]
psi['gene_id'] = psi.index.str.split("_").str[0]
psi['exon_id'] = psi.index.values

In [5]:
psi_anno = pd.merge(gtf_parsed[['gene_id', 'gene_name']], psi, on="gene_id", how="right")
psi_anno = psi_anno.set_index("exon_id").rename_axis(None)
psi_anno = psi_anno.drop(columns=["gene_id"])

### Calc. corr between ME and exon PSI

In [ ]:
corr_df = pd.DataFrame(
    columns=["Gene"] + top_qval_mods_df['Cell_type'].tolist(), 
    index=psi_anno.index
)
corr_df['Gene'] = psi_anno['gene_name'] 

for i, row in top_qval_mods_df.iterrows():
    ctype = row['Cell_type']

    mod_df = pd.read_csv(row['ME_path'])
    mod_eig = mod_df.set_index("Sample")[row['Module']]
    mod_eig = pd.to_numeric(mod_eig, errors="coerce")
    
    corrs = psi_anno.iloc[:, 1:].corrwith(mod_eig, axis=1)
    corr_df[ctype] = corrs

In [ ]:
corr_df.to_csv(f"data/tasic_2018_ALM_STAR_{psi_data}_exon_corr.csv")